In [6]:
import os

os.environ['WANDB_NOTEBOOK_NAME'] = '/Users/aditya/Programming/stock-market-predictions/stock_market_predictions/forecasting/LSTM-AllTime/MSFT-LSTM.ipynb'

In [7]:
def train():
    import pandas as pd
    from os.path import abspath

    import warnings
    warnings.filterwarnings("ignore")
    df = pd.read_csv(abspath('../../../data/MSFT/MSFT.csv'), index_col=0, parse_dates=['Date'])

    df.columns.name = df.index.name
    df.index.name = None

    df = df.filter(['Date', 'Open'])
    # Split the df into a training set and a test set
    from sklearn.model_selection import train_test_split

    # Split the data into training and testing sets
    train_set, test_set = train_test_split(df, test_size=0.05, shuffle=False)

    n_features = 1
    from sklearn.preprocessing import MinMaxScaler

    transformer = MinMaxScaler()
    train_scaled = transformer.fit_transform(train_set)
    test_scaled = transformer.fit_transform(test_set)
    
    import tensorflow as tf
    from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator # type: ignore

    n_input = 60

    train_generator=TimeseriesGenerator(train_scaled,
                                        train_scaled,
                                        length=n_input,
                                        batch_size=32)
    
    from keras.models import Sequential
    from keras.layers import Dense, LSTM

    LSTM_UNITS = 20

    model = Sequential()
    model.add(LSTM(units=LSTM_UNITS, input_shape=(n_input, n_features)))
    model.add(Dense(units=1))
    
    # Create an EarlyStopping callback
    from keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='loss', patience=5)
    
    import wandb
    from wandb.keras import WandbCallback

    wandb.init(project="stock-predictions-alltime", entity="alpheron", magic=True)

    wandb.config = {
    "learning_rate": 0.001,
    "epochs": 100,
    "batch_size": 32,
    "lstm_units": LSTM_UNITS
    }

    from keras.metrics import RootMeanSquaredError
    from stock_market_predictions.forecasting.adam import Adam

    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=[RootMeanSquaredError()])
    model.fit(train_generator, epochs=100, callbacks=[early_stopping, WandbCallback()])
    model.save(abspath('../../../logs/MSFT/'))

    inputs = df[len(df)-len(test_set)-60 : ].values
    inputs = inputs.reshape(-1,1)
    inputs = transformer.transform(inputs)
    import numpy as np

    x_test = []
    for i in range(60,len(inputs)):
        x_test.append(inputs[i-60:i, 0])
        
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    print(x_test.shape)
    predicted = model.predict(x_test)
    predicted = transformer.inverse_transform(predicted)

    # Convert the predicted values to a dataframe
    predicted_df = pd.DataFrame(predicted)

    # Rename the column to "Open"
    predicted_df.columns = ['Open']

    # Add a new column called "Date" that has the same index as the other dataframe
    predicted_df['Date'] = df[len(df)-len(test_set)-0 : ].index

    # Set the 'Date' column as the index
    predicted_df = predicted_df.set_index('Date')

    import math
    from sklearn.metrics import mean_squared_error

    def return_rmse(test, predicted):
        rmse = math.sqrt(mean_squared_error(test, predicted))
        print("the root mean squared error is: {}".format(rmse))
        return rmse

    RMSE = return_rmse(test_set, predicted)
    wandb.log({"root_mean_square_error" : RMSE})
    artifact = wandb.Artifact('Time-Series', type='model')
    artifact.add_dir(abspath('../../../logs/MSFT-5-Year/'))
    wandb.run.log_artifact(artifact)

In [8]:
sweep_config = {
    'method': 'bayes',
    'metric': 
        {
            'name': 'root_mean_square_error',
            'goal': 'minimize',
        },
    'parameters': {
        'LSTM_UNITS': {
            'values': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
        },
        'epochs': {
            'value': 100
        }
    },
    'early_terminate': {
        'type': 'hyperband',
        's': 2,
        'eta': 3,
        'max_iter': 100
    }
}

In [9]:
import wandb
# sweep_id = wandb.sweep(sweep_config, project="stock-predictions-alltime", entity="alpheron")

In [10]:
wandb.agent('8gz0pdfk', function=train, project="stock-predictions-alltime", entity="alpheron")

wandb: Agent Starting Run: hcov2hco with config:
wandb: 	LSTM_UNITS: 15
wandb: 	epochs: 100
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/100
73/73 [==============================] - 18s 238ms/step - loss: 0.0076 - root_mean_squared_error: 0.0874
Epoch 2/100
73/73 [==============================] - 21s 292ms/step - loss: 2.3313e-04 - root_mean_squared_error: 0.0153
Epoch 3/100
73/73 [==============================] - 17s 228ms/step - loss: 2.9401e-04 - root_mean_squared_error: 0.0171
Epoch 4/100
73/73 [==============================] - 13s 177ms/step - loss: 2.5309e-04 - root_mean_squared_error: 0.0159
Epoch 5/100
73/73 [==============================] - 13s 178ms/step - loss: 2.3125e-04 - root_mean_squared_error: 0.0152
Epoch 6/100
73/73 [==============================] - 15s 208ms/step - loss: 2.2609e-04 - root_mean_squared_error: 0.0150
Epoch 7/100
73/73 [==============================] - 14s 197ms/step - loss: 2.2866e-04 - root_mean_squared_error: 0.0151
Epoch 8/100
73/73 [==============================] - 14s 188ms/step - loss: 2.6189e-04 - root_mean_squared_error: 0.0162
Epoch 9/100
73/73 [=================

INFO:tensorflow:Assets written to: /Users/aditya/Programming/stock-market-predictions/logs/MSFT/assets


INFO:tensorflow:Assets written to: /Users/aditya/Programming/stock-market-predictions/logs/MSFT/assets


(126, 60, 1)


...

(126, 60, 1)


wandb: Adding directory to artifact (/Users/aditya/Programming/stock-market-predictions/logs/MSFT-5-Year)... Done. 0.0s


the root mean squared error is: 9.516257608295364


epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_square_error,▁
root_mean_squared_error,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,20
loss,0.00027
root_mean_square_error,9.51626
root_mean_squared_error,0.01632


wandb: Agent Starting Run: 5k6tlo1d with config:
wandb: 	LSTM_UNITS: 40
wandb: 	epochs: 100
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.
